# 发现关联规则

客户在餐厅点餐时，面对菜单中大量的菜品信息，往往无法迅速找到满意的菜品，既增加了点菜的时间，也降低了客户的就餐体验。实际上，菜品的合理搭配是有规律可循的：顾客的饮食习惯、菜品的荤素和口味，都是影响菜品搭配关键因素。有些菜品之间是相互关联的，而有些菜品之间是对立或竞争关系（负关联），这些规律都隐藏在大量的历史菜单数据中，如果能够通过数据挖掘发现客户点餐的规则，就可以快速识别客户的口味，当他下了某个菜品的订单时，推荐相关联的菜品，引导客户消费，提高顾客的就餐体验和餐饮企业的业绩水平。

关联规则分析也称为购物篮分析，最早是为了发现超市销售数据库中不同的商品之间的关联关系。例如：一个超市的经理想要更多的了解顾客的购物习惯，比如：“哪组商品可能会在一次购物中同时购买？”或者“某顾客购买了个人电脑，那该顾客三个月后购买数码相机的概率有多大？”他可能会发现如果购买了面包的顾客同时非常有可能会购买牛奶，这就推导出了一条关联规则“面包->牛奶”，其中面包称为规则的前项，而牛奶称为后项。通常对面包降低售价进行促销，而适当提高牛奶的售价，关联销售出的牛奶就有可能增加超市的整体利润。

关联规则分析是数据挖掘中最活跃的研究方法之一，目的是在一个数据集中找出各项之间的关联关系，而这种关系并没有在数据中直接表示出来。

## 1. 关联规则相关的定义

关联规则是针对频繁出现的项集（Itemset，同时出现的项的集合）来进行分析的，针对项集有几个比较重要的度量。


### 1.1. 支持度
项集A和项集B同时发生的概率称为关联规则的支持度，也称为相对支持度。

Support(A,B) = P(A,B) = A与B同时出现的事务次数 / 所有事务次数

### 1.2. 置信度
项集A出现，则项集B出现的概率为关联规则的置信度。

Confidence(A,B) = P(B|A) = Support(A,B) / Support(A)

### 1.3. 提升度
项集A与项集B之间的提升度，指出了项集A，B之间是否相关，通常提升度大于1则说明两个项集之间互相依赖，并不独立。

Lift(A,B) = Support(A,B) / Support(A) * Support(B)

### 1.4. Conviction度
Conviction(A,B) = (1 - Support(B)) / (1 - Confidence(A,B))

## 2. 关联规则算法

### 2.1. 常用关联规则挖掘算法
1. Aprior：关联规则最常用的挖掘频繁项集的算法，其核心思想是通过连接产生候选项及其支持度然后通过剪枝生成频繁项集
2. FP-Tree：针对Aprior算法的固有的多次扫描事务数据集的缺陷，提出的不产生候选频繁项集的方法。
3. Eclat：是一种深度优先的算法，采用垂直数据表示形式，在概念格理论的基础上利用基于前缀的等价关系将搜索空间划分为较小的子空间。
4. 灰色关联法：分析和确定各因素之间的影响程度或是若干个子因素对主因素的贡献度而形成的一种分析方法

### 2.2. 其他一些计算相关度的方法
1. 互信息：通过互信息可以有效度量两个商品频繁共现的信息增益，来计算是否两个项集有密切关系。
2. t检验：通过统计两个项集的贡献频率来检测两个项集是否独立。

### 2.3. Apriori算法
通过设定最小支持度和最小置信度两个阈值，如果某两个项集的置信度和支持度都超过了设定的阈值的时候，则此项集可以描述成是很强的关联规则。

In [1]:
import pandas as pd
import numpy as np

input_file = '../data/menu_orders.xls'
data = pd.read_excel(input_file, header = None)
data.head()

,0,1,2,3
0,a,c,e,NaN
1,b,d,NaN,NaN
2,b,c,NaN,NaN
3,a,b,c,d
4,a,b,NaN,NaN


In [2]:
import math
from apyori import apriori

filter_nan = lambda lst: list(filter(lambda x: type(x) is str, lst))
transactions = list(map(lambda lst: filter_nan(lst), data.values.tolist()))

min_support = 0.2
min_confidence = 0.5
min_lift = 0.8

results = list(apriori(transactions, min_support = min_support, min_confidence = min_confidence, min_lift = min_lift))
def display_apriori_result(results):
    for r in results:
        items = set(r.items)
        if len(items) > 1:
            print('items: %s, support: %f' % (items, r.support))
            for different_order in r.ordered_statistics:
                order = list(different_order.items_base) + list(different_order.items_add)
                confidence = different_order.confidence
                lift = different_order.lift
                print('  %s, confidence: %.3f, lift: %.3f' % (order, confidence, lift))

display_apriori_result(results)

items: {'b', 'a'}, support: 0.500000
  ['a', 'b'], confidence: 0.714, lift: 0.893
  ['b', 'a'], confidence: 0.625, lift: 0.893
items: {'c', 'a'}, support: 0.500000
  ['a', 'c'], confidence: 0.714, lift: 1.020
  ['c', 'a'], confidence: 0.714, lift: 1.020
items: {'e', 'a'}, support: 0.300000
  ['e', 'a'], confidence: 1.000, lift: 1.429
items: {'c', 'b'}, support: 0.500000
  ['b', 'c'], confidence: 0.625, lift: 0.893
  ['c', 'b'], confidence: 0.714, lift: 0.893
items: {'b', 'd'}, support: 0.200000
  ['d', 'b'], confidence: 1.000, lift: 1.250
items: {'c', 'e'}, support: 0.300000
  ['e', 'c'], confidence: 1.000, lift: 1.429
items: {'c', 'b', 'a'}, support: 0.300000
  ['b', 'a', 'c'], confidence: 0.600, lift: 0.857
  ['c', 'b', 'a'], confidence: 0.600, lift: 0.857
items: {'c', 'e', 'a'}, support: 0.300000
  ['c', 'a', 'e'], confidence: 0.600, lift: 2.000
  ['e', 'a', 'c'], confidence: 1.000, lift: 1.429
  ['c', 'e', 'a'], confidence: 1.000, lift: 1.429


#### 解读找到的关联规则
就第一条输出结果进行解释：客户同时点菜品a和b的概率是50%，如果点了菜品a，再点菜品b的概率是71.4%。如果点了菜品b，再点菜品a的概率是62.5%。

如果点菜人员知道了这些，就可以对顾客进行智能推荐，增加销量同时满足客户需求。

## 3. 中医症状与乳腺癌诊断的关联规则分析

中医通常有毒副作用小的特点，同时有调理身体的作用，做到防未病的作用。通过分析癌症患者的不同阶段的不同情况，找到中医症状之间的关联关系，可以尝试做到在后续症候尚未出现之前，就可以进行哪些介入的手段，组织病情的恶化。

在本次案例中，通过分析乳腺癌患者的中医症状数据，发现：
* 当肝气郁结证型处于第三阶段A3，肝肾阴虚证型处于第四阶段F4的时候，乳腺癌处于4期的可能性非常高，达到88.1%的概率。
* 当冲任失调证型处于第三阶段C3，肝肾阴虚证型处于第四阶段F4的时候，乳腺癌处于4期的可能性也很高，达到87.5%的概率。
* 当热毒蕴结证型达到第二阶段B2，肝肾阴虚证型处于第四阶段F4的时候，乳腺癌处于4期的可能性非常高，达到79.45%的概率。

综上分析，乳腺癌进入4期的患者证型主要为肝肾阴虚、热毒蕴结、肝气郁结和冲任失调。

模型结果表明，针对乳腺癌患者应该关注肝肾阴虚证型的进展，当出现相关证型的表现时，可以炫耀滋补肝肾、清热解毒类抗癌中药。阻止病情继续进展，为患者接受进一步治疗争取机会。同时在进行治疗时须本着身心一体，综合治疗的精神，重视心里调适。
* 一方面在药方中注重疏肝解郁
* 另一方面需要及时疏导患者抑郁、焦虑的不良情绪

### 3.1. 对中医症状进行数据处理

通过关联分析，发现中医症状和乳腺癌的关系，需要对数据进行处理，因为Aprori数据只能对离散化的数据进行分析。

首先我们加载数据查看一下

In [3]:
from sklearn.cluster import KMeans #导入K均值聚类算法

input_file = '../data/chinese_medicine_diagnosis_data.xls' #待聚类的数据文件

#读取数据并进行聚类分析
data = pd.read_excel(input_file) #读取数据
data.head()

,肝气郁结证型系数,热毒蕴结证型系数,冲任失调证型系数,气血两虚证型系数,脾胃虚弱证型系数,肝肾阴虚证型系数,病程阶段,TNM分期,转移部位,确诊后几年发现转移
0,0.056,0.460,0.281,0.352,0.119,0.350,S4,H4,R1,J1
1,0.488,0.099,0.283,0.333,0.116,0.293,S4,H4,R1,J1
2,0.107,0.008,0.204,0.150,0.032,0.159,S4,H4,R2,J2
3,0.322,0.208,0.305,0.130,0.184,0.317,S4,H4,R2,J1
4,0.242,0.280,0.131,0.210,0.191,0.351,S4,H4,R2R5,J1


数据中包含有“肝气郁结证型系数”、“热毒蕴结证型系数”、“冲任失调证型系数”、“气血两虚证型系数”、“脾胃虚弱证型系数”、“肝肾阴虚证型系数”这六类中医证型。

通过聚类的方法可以将具体的证型系统转换成离散值，然后就可以通过Apriori算法对数据进行分析。

In [4]:
typelabel ={u'肝气郁结证型系数':'A', u'热毒蕴结证型系数':'B', u'冲任失调证型系数':'C', u'气血两虚证型系数':'D', u'脾胃虚弱证型系数':'E', u'肝肾阴虚证型系数':'F'}
k = 4 #需要进行的聚类类别数
keys = list(typelabel.keys())

result = pd.DataFrame()
new_data = pd.DataFrame()
for i in range(len(keys)):
    #调用k-means算法，进行聚类离散化
    print(u'正在进行“%s”的聚类...' % keys[i])
    kmodel = KMeans(n_clusters = k, n_jobs = 4) #n_jobs是并行数，一般等于CPU数较好
    input_data = data[[keys[i]]].as_matrix()
    kmodel.fit(input_data) #训练模型，是否可以尝试GMM聚类
    #labels = pd.Series(kmodel.labels_).map(lambda x: (typelabel[keys[i]]+'{}').format(x+1))
    #new_data = pd.concat([new_data, labels], axis=1)
    #print(input_data[:5])
    #print(labels.head())

    r1 = pd.DataFrame(kmodel.cluster_centers_, columns = [typelabel[keys[i]]]) #聚类中心
    r2 = pd.Series(kmodel.labels_).value_counts() #分类统计
    r2 = pd.DataFrame(r2, columns = [typelabel[keys[i]]+'_num']) #转为DataFrame，记录各个类别的数目
    r = pd.concat([r1, r2], axis = 1).sort_values(by = typelabel[keys[i]]) #匹配聚类中心和类别数目
    r.index = [1, 2, 3, 4]

    r[typelabel[keys[i]]] = pd.rolling_mean(r[typelabel[keys[i]]], 2) #rolling_mean()用来计算相邻2列的均值，以此作为边界点。
    r[typelabel[keys[i]]][1] = 0.0 #这两句代码将原来的聚类中心改为边界点。
    result = result.append(r.T)
    
    labels = input_data - r[typelabel[keys[i]]].values
    labels = np.greater(labels, np.zeros(labels.shape)).astype(int).sum(axis=1)
    labels = pd.Series(labels).map(lambda x: (typelabel[keys[i]]+'{}').format(x))
    new_data = pd.concat([new_data, labels], axis=1)

print(result)

new_data = pd.concat([new_data, data.iloc[:,7]], axis=1)
new_data.columns = list(typelabel.keys()) + [u'TNM分期']

正在进行“肝气郁结证型系数”的聚类...
正在进行“热毒蕴结证型系数”的聚类...


/Users/yingrui/miniconda3/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:24: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=2,center=False).mean()
/Users/yingrui/miniconda3/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


正在进行“冲任失调证型系数”的聚类...
正在进行“气血两虚证型系数”的聚类...
正在进行“脾胃虚弱证型系数”的聚类...
正在进行“肝肾阴虚证型系数”的聚类...
           1           2           3           4
A        0.0    0.178698    0.257724    0.351843
A_num  240.0  356.000000  281.000000   53.000000
B        0.0    0.153543    0.298217    0.489954
B_num  342.0  380.000000  179.000000   29.000000
C        0.0    0.202149    0.289061    0.423537
C_num  297.0  394.000000  204.000000   35.000000
D        0.0    0.172281    0.251683    0.359353
D_num  283.0  375.000000  228.000000   44.000000
E        0.0    0.152698    0.257873    0.376062
E_num  273.0  319.000000  245.000000   93.000000
F        0.0    0.179143    0.261386    0.354643
F_num  200.0  237.000000  265.000000  228.000000


### 3.2. 对中医症状进行关联分析

经过聚类的结果对数据进行标记，将数据离散化的结果如下：

In [5]:
new_data.head()

,肝气郁结证型系数,热毒蕴结证型系数,冲任失调证型系数,气血两虚证型系数,脾胃虚弱证型系数,肝肾阴虚证型系数,TNM分期
0,A1,B3,C2,D3,E1,F3,H4
1,A4,B1,C2,D3,E1,F3,H4
2,A1,B1,C2,D1,E1,F1,H4
3,A3,B2,C3,D1,E2,F3,H4
4,A2,B2,C1,D2,E2,F3,H4


In [6]:
support = 0.06
confidence = 0.75 #最小置信度

records = list(new_data.values.tolist())
association_rules = list(apriori(records, 
                                 min_support = support, 
                                 min_confidence = confidence, 
                                 min_lift=1))

display_apriori_result(association_rules)

items: {'D2', 'A2', 'E3'}, support: 0.072043
  ['A2', 'E3', 'D2'], confidence: 0.753, lift: 1.867
items: {'F4', 'A3', 'H4'}, support: 0.079570
  ['F4', 'A3', 'H4'], confidence: 0.881, lift: 1.974
items: {'F4', 'C3', 'H4'}, support: 0.075269
  ['F4', 'C3', 'H4'], confidence: 0.875, lift: 1.961
items: {'D2', 'A2', 'F3', 'H4'}, support: 0.062366
  ['D2', 'F3', 'H4', 'A2'], confidence: 0.753, lift: 1.968


## 4. 总结
- 规则基本形式：如果 A 发生  ==> B(更可能)发生。我们经常在陈述的时候都忽略了“更可能的”，但同时如果要陈述更可能的时候，要注意规则并非确定无疑（是否存在因果关系？计算结果的前项和后项不代表因果关系）
- 啤酒和尿布故事，是算法很好的营销，但是算法价值被放大了，推荐引擎不只是关联分析。
- 交叉销售的价值点主要在于长尾，而不是"big surprise"

## 作业

1. 针对购物篮数据进行关联规则分析（数据文件：../data/shopping_basket.csv）